In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import os
import wandb
from wandb.keras import WandbCallback

import Sensory_GRU

In [102]:
def train():
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    import matplotlib.pyplot as plt
    import numpy as np
    import wandb
    from wandb.keras import WandbCallback

    def seq_acc(y_true, y_pred):
        y_bin=np.zeros_like(y_pred)
        for i, dd in enumerate(y_bin):
            for j in range(len(dd)):
                pred=y_pred[i][j]
                if pred>=0.5:
                    y_bin[i][j]=1
                else:
                    y_bin[i][j]=0

        predict_true = (y_true == y_bin)
        # multi-column일 경우에도 계산 할 수 있도록 np.average를 한번 더 씌움
        try:
            score = np.average(np.average(predict_true))
        except ValueError:
            score = mean_squared_error(y_true, y_bin)
        return score

    def my_seq_acc(y_true, y_pred):
        score = tf.py_function(func=seq_acc, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_seq_acc') # tf 2.x
        #score = tf.py_func( lambda y_true, y_pred : mse_AIFrenz(y_true, y_pred) , [y_true, y_pred], 'float32', stateful = False, name = 'custom_mse' ) # tf 1.x
        return score


    class MySeqAccCallback(keras.callbacks.Callback):
        def on_epoch_end(self, epochs, logs=None):
            y_pred=self.model.predict(X_test)
            print('sequence accuracy is {}'.format(seq_acc(y_test, y_pred)))


    default_config={
                         'seq_field':72,
                         'stride_inside_seq':9,
                         'stride_between_seqs':2,
                         'learning_rate':0.01,
                         'split_train_ratio':0.8,
                         'epochs':20,
                         'batch_size':64,
                         'unit_gru0':64}
    wandb.init(config = default_config)

    locations=['거문도', '울산', '거제도', '통영', '추자도']

    # load normalized data

    df_merged=pd.read_csv("sensory_preprocessed_df.csv")
    if df_merged.columns[0]=='Unnamed: 0':
        df_merged = df_merged.iloc[:, 1:]

    print('loaded dataset. Generating sequences')
    seq_length=wandb.config.seq_field//wandb.config.stride_inside_seq
    len_ds=len(df_merged)

    seqs_idx=[]

    start_idx=0
    while start_idx<=len_ds-wandb.config.seq_field:
        seqs_idx.append(list(range(start_idx, start_idx + wandb.config.seq_field, wandb.config.stride_inside_seq
    )))
        start_idx+=wandb.config.stride_between_seqs


    seqs_idx[100],len(seqs_idx[100])

    df_merged.reset_index(inplace=True, drop=True)
    print('Any missing values exist:', df_merged.isna().all().all())


    #train_cols=['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '적조발생']
    ds_train_cols=df_merged
    ds_train_cols.reset_index(inplace=True, drop=True)
    print('train dataset columns:',ds_train_cols.columns)

    seq_dataset=np.zeros([len(seqs_idx), len(seqs_idx[0]), len(ds_train_cols.columns)])

    for i, seq in enumerate(seqs_idx):
        for j, row_number in enumerate(seq):
            seq_dataset[i, j]=ds_train_cols.loc[row_number].to_numpy()

    def not_bin_in_occurence(x):
        if x==1 or x==0:
            return x
        else:
            print('exceptional value(not 0 or 1) found. replaced by near one.')
            if x>=0.5:
                return 1
            else:
                return 0
            
    ds_train_cols['적조발생']=ds_train_cols['적조발생'].apply(not_bin_in_occurence)


    split_index=int(len(seq_dataset)*wandb.config.split_train_ratio)
    print(split_index, len(seq_dataset))

    train_xy=seq_dataset[:split_index]
    np.random.shuffle(train_xy)
    X_train=train_xy[:,:,0:-1]
    y_train=train_xy[:,:,-1]

    test_xy=seq_dataset[split_index:]
    np.random.shuffle(test_xy)
    X_test=test_xy[:,:,0:-1]
    y_test=test_xy[:,:,-1]

    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape,'\n\n')

    model = keras.Sequential([
        keras.Input(shape=(seq_length, 25)),
        keras.layers.GRU(wandb.config.unit_gru0),
        keras.layers.Dense(seq_length, activation="sigmoid"),
    ]
    )


    # In[95]:
    optimizer=keras.optimizers.Adam(learning_rate=wandb.config.learning_rate)

    model.compile(optimizer=optimizer, loss="binary_crossentropy")

    # In[97]:


    # In[98]:


    model.fit(X_train, y_train,
            batch_size=wandb.config.batch_size,
            epochs=wandb.config.epochs, 
            validation_data=(X_test, y_test),
            callbacks=[WandbCallback(training_data = (X_train, y_train),
                                     validation_data = (X_test, y_test),
                                     log_weights = True,
                                     log_gradients = True), MySeqAccCallback()])
    y_pred=model.predict(X_test)
    wandb.log({"Validation Acc :":seq_acc(y_test, y_pred)})

In [103]:
wandb.login()

True

In [104]:
# yaml 파일로 만들어도 됩니다.

sweep_config = {
    "name": "sweep3",
    "metric": {
        "goal": "minimize",
        "name": "val_loss"
    },
    "method": "random",
    "parameters": {
        "split_train_ratio":{
            "values":[0.5,0.8]
        },
        "epochs":{
            "values":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
        },
        "learning_rate":{'max':0.01, 'min':0.0001},
        "unit_gru0":{"values":[32,64,128,256]},
        "stride_inside_seq":{"values":[1,3,9]}
    }
    }

In [ ]:
sweep_id = wandb.sweep(sweep_config)

# run the sweep
wandb.agent(sweep_id,
            function=train,
            entity = 'chhyyi',
            project = 'redzone_gru_sweep')

Create sweep with ID: 5qc8l882
Sweep URL: https://wandb.ai/chhyyi/redzone_gru_sweep/sweeps/5qc8l882


wandb: Agent Starting Run: y1ubyamg with config:
wandb: 	epochs: 6
wandb: 	learning_rate: 0.00534501159529119
wandb: 	split_train_ratio: 0.8
wandb: 	stride_inside_seq: 3
wandb: 	unit_gru0: 32


loaded dataset. Generating sequences
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생'],
      dtype='object')
20106 25133
(20106, 24, 25) (5027, 24, 25) (20106, 24) (5027, 24) 


Epoch 1/6
315/315 [==============================] - 3s 5ms/step - loss: 0.1525 - val_loss: 0.1932


wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221113_092732-y1ubyamg/files/model-best)... Done. 0.0s


sequence accuracy is 0.9182166301969366
Epoch 2/6
315/315 [==============================] - 1s 4ms/step - loss: 0.0615 - val_loss: 0.1795


...

Validation Acc :,▁
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
loss,█▅▃▂▂▁▁▁▂▁▁▁▁▁▁▁
val_loss,▁▃▄▄▅▅▇▇▆▇███▇▆▇
Validation Acc :,0.92262
best_epoch,0
best_val_loss,0.38154
epoch,15
loss,0.00643
val_loss,0.94793


wandb: Agent Starting Run: s9txzh6o with config:
wandb: 	epochs: 15
wandb: 	learning_rate: 0.008869358439563687
wandb: 	split_train_ratio: 0.5
wandb: 	stride_inside_seq: 1
wandb: 	unit_gru0: 32


loaded dataset. Generating sequences
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생'],
      dtype='object')
12566 25133
(12566, 72, 25) (12567, 72, 25) (12566, 72) (12567, 72) 


Epoch 1/15
197/197 [==============================] - 3s 10ms/step - loss: 0.1354 - val_loss: 0.2836


wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221113_103601-s9txzh6o/files/model-best)... Done. 0.0s


sequence accuracy is 0.9263138466707338
Epoch 2/15
197/197 [==============================] - 1s 7ms/step - loss: 0.0411 - val_loss: 0.4354
sequence accuracy is 0.9233884158687214
Epoch 3/15
197/197 [==============================] - 1s 7ms/step - loss: 0.0179 - val_loss: 0.5311
sequence accuracy is 0.9145502329734844
Epoch 4/15
197/197 [==============================] - 1s 7ms/step - loss: 0.0128 - val_loss: 0.5577
sequence accuracy is 0.9284468581735232
Epoch 5/15
197/197 [==============================] - 1s 7ms/step - loss: 0.0140 - val_loss: 0.5756
sequence accuracy is 0.9301377947534548
Epoch 6/15
197/197 [==============================] - 1s 7ms/step - loss: 0.0058 - val_loss: 0.6042
sequence accuracy is 0.9254208553265607
Epoch 7/15
197/197 [==============================] - 1s 7ms/step - loss: 0.0062 - val_loss: 0.6333


In [3]:
wandb.finish()

# GRU 써보기    
이 노트북은 [Sensory_LSTM 노트북](https://github.com/chhyyi/aiffel/blob/main/aiffelthon/Sensory_LSTM.ipynb) 의 복제입니다.

## 작업 로그

### 2022-11-12
- 지난번에 하려고 했던, wandb를 이용해서 sweep을 해봅니다. wandb에 의한 sweep은 python module로 저장하고 임포트 하는 것이 더 적당해 보입니다. 이 부분은 천천히 생각합니다. 아무튼 그 경우, '\*.py' 파일을 내보내고 여기에서 wandb sweep 을 실행하도록 합니다.  

### 2022-11-10
- 원본의 LSTM을 GRU로 바꾸고, wandb를 써보려고 했는데 그냥 한 번 했습니다. 
- df_merged를 csv로 저장하고, 시퀀스 인덱스 생성 이전의 부분은 모두 삭제합니다. csv파일을 읽는 부분부터 씁니다.

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import wandb
from wandb.keras import WandbCallback

In [18]:
wandb.login()

True

In [19]:
run = wandb.init(project = 'redzone_gru',
                 entity = 'chhyyi',
                 config = {
                     'seq_field':72,
                     'stride_inside_seq':9,
                     'stride_between_seqs':2,
                     'learning_rate':0.01,
                     'loss':'sparse_categorical_crossentropy',
                     'metrics':['accuracy'],
                     'epochs':20,
                     'batch_size':64,
                     'callbacks'=[MySeqAccCallback()]
                 })

SyntaxError: invalid syntax (2143752845.py, line 12)

In [ ]:
locations=['거문도', '울산', '거제도', '통영', '추자도']

In [93]:
# load normalized data

df_merged=pd.read_csv("sensory_preprocessed_df.csv")
df_merged

,Unnamed: 0,풍속(m/s),풍향(deg),기온(°C),수온(°C),강수량(mm),풍속(m/s).1,풍향(deg).1,기온(°C).1,수온(°C).1,...,풍향(deg).3,기온(°C).3,수온(°C).3,강수량(mm).3,풍속(m/s).4,풍향(deg).4,기온(°C).4,수온(°C).4,강수량(mm).4,적조발생
0,0,-0.846358,0.903430,-0.460595,-0.297487,-0.124668,-1.539878,0.752017,-0.658575,-0.665820,...,0.586254,-0.442772,-0.243601,-0.150869,-1.353779,1.551810,-0.364254,-0.500266,-0.112367,0.0
1,1,-0.308964,0.861089,-0.404756,-0.276128,-0.124668,-1.661737,0.652532,-0.588377,-0.620489,...,0.638984,-0.449676,-0.243601,-0.150869,-0.406414,-1.337830,-0.391642,-0.519119,-0.112367,0.0
2,2,-0.428385,0.945770,-0.348918,-0.297487,-0.124668,-1.448484,0.534960,-0.616456,-0.643154,...,0.691715,-0.456580,-0.243601,-0.150869,-0.161932,1.378432,-0.377948,-0.519119,-0.112367,0.0
3,3,-0.338819,0.827217,-0.293080,-0.404279,-0.124668,-1.539878,0.652532,-0.546258,-0.665820,...,0.709292,-0.428963,-0.243601,-0.150869,-0.559215,1.386688,-0.323173,-0.519119,-0.112367,0.0
4,4,0.168719,1.064324,-0.320999,-0.468355,-0.124668,0.074754,0.842458,-0.447980,-0.688485,...,0.753234,-0.401346,-0.262635,-0.150869,0.113109,-1.329574,-0.336867,-0.500266,-0.112367,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50331,50331,-1.712158,0.115895,0.879527,1.218967,-0.124668,-1.082907,-0.405622,0.899828,1.260715,...,0.419274,1.062370,1.222006,-0.150869,-1.537140,-1.329574,0.881887,1.196504,-0.112367,0.0
50332,50332,-1.353896,0.183640,0.893486,1.176250,-0.124668,-0.991512,-0.360402,0.857709,1.238050,...,0.392909,1.103796,1.241040,-0.150869,-1.384339,-1.222244,0.553234,1.026827,-0.112367,0.0
50333,50333,-1.353896,0.454619,0.879527,1.176250,-0.124668,-1.143836,-0.360402,0.857709,1.215385,...,0.805964,1.062370,1.126837,-0.150869,-1.567700,1.477505,0.347826,0.970268,-0.112367,0.0
50334,50334,-1.234475,0.573173,0.837648,1.261684,-0.124668,-1.418019,-0.043860,0.815590,1.215385,...,0.762022,1.034753,1.088769,-0.150869,-1.567700,1.361919,0.690173,0.932562,-0.112367,0.0


In [94]:
# drop surplus columns, if required
df_merged = df_merged.iloc[:, 1:]


In [96]:
df_merged.isna().all()

풍속(m/s)      False
풍향(deg)      False
기온(°C)       False
수온(°C)       False
강수량(mm)      False
풍속(m/s).1    False
풍향(deg).1    False
기온(°C).1     False
수온(°C).1     False
강수량(mm).1    False
풍속(m/s).2    False
풍향(deg).2    False
기온(°C).2     False
수온(°C).2     False
강수량(mm).2    False
풍속(m/s).3    False
풍향(deg).3    False
기온(°C).3     False
수온(°C).3     False
강수량(mm).3    False
풍속(m/s).4    False
풍향(deg).4    False
기온(°C).4     False
수온(°C).4     False
강수량(mm).4    False
적조발생         False
dtype: bool


## 시퀀스로 전환
얼마의 길이로 자를 것인가 하는게 중요한 hyper-parameter가 될 수도 있겠습니다. 1시간이 1timestep입니다. 만약 다음과 같이 한다면 sequence의 구조는 이렇게 됩니다.  
- 한 시퀀스의 field: 72
- window size: 1
- stride between elements of sequence: 6 
- stride between sequences: 7
- sequence length $\frac{72}{6}=12$  
- 
그러면 1번째 시퀀스는 timestep=0, 2번째 시퀀스는 timtestep=7에서 시작합니다. 그렇게 하면 시퀀스의 배치가 이렇게 됩니다.
|timestep|0|1|2|3|4|5|6|7|8|9|1|11|12|13|14|15|16|17|18|19|20|21|22|23|24|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|-|
|seq\[0\]|1||||||2||||||3||||||4||||||5|
|seq\[1\]||||||||1||||||2||||||3||||||
|seq\[2\]|||||||||||||||1||||||2|||||

또한, 전체 시퀀스가 시작되거나 끝나는 부분이 아닌 곳에서는 하나의 데이터 포인트가 두 개의 시퀀스에 포함되게 됩니다.

## 모델의 입력과 출력
입력은 일단 5개 지점의 5개 데이터를 통합, 인코더의 embedding size가 $5\times5=25$인 모델로 작동합니다.

In [22]:
wandb.config.seq_field
wandb.config.stride_inside_seq
wandb.config.stride_between_seqs
seq_length=seq_field//stride
len_ds=len(df_merged)

seqs_idx=[]

start_idx=0
while start_idx<=len_ds-wandb.config.seq_field:
    seqs_idx.append(list(range(start_idx, start_idx + wandb.config.seq_field, wandb.config.stride_inside_seq
)))
    start_idx+=wandb.config.stride_between_seqs

Error: You must call wandb.init() before wandb.config.seq_field

생성된 seqs_idx를 이미지로 바꿔서 한번 살펴봅니다.

In [ ]:
seqs_idx[100],len(seqs_idx[100])

In [ ]:
img_size_x = 263 #usually, match with seq_idx[-1][-1]
img_size_y = 100 #number of sequence to visuallize
seq_length_squeeze_ratio = 1

img = np.zeros([img_size_y, img_size_x])

for yidx, seq in enumerate(seqs_idx[:img_size_y]):
    #for i in range(seq[0]//seq_length_squeeze_ratio,seq[-1]//seq_length_squeeze_ratio + 1):
    for i in seq:
        if i<img_size_x:
            img[yidx, i]=1.0

plt.figure(figsize=(12,4))
plt.xticks(ticks=list(range(0,img_size_x,seq_field)))
plt.yticks(ticks=list(range(0,img_size_y,10)))

plt.grid(alpha=0.5)
plt.imshow(img)
plt.colorbar()
#plt.colorbar(location='bottom')
plt.show()

figure는 50행(수직축)까지 시퀀스별 time-index(수평축)를 점으로 표시합니다.

## train, test set 구성
먼저 index를 리셋하고 시작합니다..

In [ ]:

df_merged.reset_index(inplace=True, drop=True)
#ds0=ds0[df_default_index]
print(df_merged.isna().any().any())
df_merged

In [ ]:
df_merged.columns

In [ ]:
#train_cols=['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '적조발생']
ds_train_cols=df_merged
ds_train_cols.reset_index(inplace=True, drop=True)
ds_train_cols

In [ ]:
seq_dataset=np.zeros([len(seqs_idx), len(seqs_idx[0]), len(ds_train_cols.columns)])
seq_dataset.shape

In [ ]:
for i, seq in enumerate(seqs_idx):
    for j, row_number in enumerate(seq):
        seq_dataset[i, j]=ds_train_cols.loc[row_number].to_numpy()

seq_dataset[:2]

In [ ]:
ds_train_cols.loc[0].to_numpy()

### label 예외처리: 
이유를 찾지 못했는데 라벨에 0, 1아닌 값들이 있어 임시로 씁니다.

In [ ]:
def not_bin_in_occurence(x):
    if x==1 or x==0:
        return x
    else:
        print('exceptional value(not 0 or 1) found. replaced by near one.')
        if x>=0.5:
            return 1
        else:
            return 0
        
ds_train_cols['적조발생']=ds_train_cols['적조발생'].apply(not_bin_in_occurence)

### train - test 셋 분리 :
특정 연도들에 대한 과적합을 없애기 위해 test와 train을 섞지 않고 시계열에서 일단 나눕니다. 그 이후 shuffle합니다.  
셔플 참고자료(stack overflow) https://stackoverflow.com/questions/35646908/numpy-shuffle-multidimensional-array-by-row-only-keep-column-order-unchanged

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(seq_dataset[:,:,0:-1], seq_dataset[:,:,-1], test_size=0.20, random_state=42)

#1. 8:2 split
split_index=int(len(seq_dataset)*0.8)
print(split_index, len(seq_dataset))

#2. 5:5 split
#split_index=int(len(ds_train_cols)*0.5)
train_xy=seq_dataset[:split_index]
np.random.shuffle(train_xy)
X_train=train_xy[:,:,0:-1]
y_train=train_xy[:,:,-1]

test_xy=seq_dataset[split_index:]
np.random.shuffle(test_xy)
X_test=test_xy[:,:,0:-1]
y_test=test_xy[:,:,-1]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape,'\n\n')


In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape,'\n\n')
print('print data for one sequence')

# model & train
간단한 GRU 모델을 훈련해봅니다.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
#copied from:
#https://www.dacon.io/competitions/official/235584/codeshare/738
def seq_acc(y_true, y_pred):
    y_bin=np.zeros_like(y_pred)
    for i, dd in enumerate(y_bin):
        for j in range(len(dd)):
            pred=y_pred[i][j]
            if pred>=0.5:
                y_bin[i][j]=1
            else:
                y_bin[i][j]=0
            
    predict_true = (y_true == y_bin)
    # multi-column일 경우에도 계산 할 수 있도록 np.average를 한번 더 씌움
    try:
        score = np.average(np.average(predict_true))
    except ValueError:
        score = mean_squared_error(y_true, y_bin)
    return score

def my_seq_acc(y_true, y_pred):
    score = tf.py_function(func=seq_acc, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_seq_acc') # tf 2.x
    #score = tf.py_func( lambda y_true, y_pred : mse_AIFrenz(y_true, y_pred) , [y_true, y_pred], 'float32', stateful = False, name = 'custom_mse' ) # tf 1.x
    return score


In [ ]:

model = keras.Sequential([
    keras.Input(shape=(8, 25)),
    layers.GRU(64, return_sequences=True),
    layers.GRU(64),
    layers.Dense(8, activation="sigmoid"),
]
)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy")

In [ ]:
model.summary()

In [ ]:
class MySeqAccCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs=None):
        y_pred=self.model.predict(X_test)
        print('sequence accuracy is {}'.format(seq_acc(y_test, y_pred)))

In [ ]:
history = model.fit(X_train, y_train, 
                    batch_size=32, 
                    epochs=10, 
                    validation_data=(X_test, y_test),
                    callbacks=[MySeqAccCallback()],
                   )

In [ ]:
#print(history.history['loss'])
#print(history.history['val_loss'])

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
y_pred=model.predict(X_test)
print(seq_acc(y_test, y_pred))

In [ ]:
del model